In [1]:
import numpy as np

In [2]:
import pandas as pd
import pickle as pkl

In [3]:
from sklearn.preprocessing import OneHotEncoder

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
from scipy.stats import spearmanr, kendalltau

Для начала прочитаем и, для более простого анализа, преобразуем исходные данные в `pandas.DataFrame`.

## Игроки

In [6]:
players_df = pd.DataFrame().from_dict(pkl.load(open('players.pkl', 'rb'))).T
players_df.sample(4)

,id,name,patronymic,surname
63678,63678,Антон,Олегович,Минашкин
45549,45549,Дмитрий,,Новиков
161753,161753,Фазиль,,Бабаев
107765,107765,Дмитрий,,Седельников


Как видно, таблица `players` содержит только информацию об идентификаторе, имени, отчестве (если имеется) и фамилии игрока. В дальнейшем, думается, можно будет использовать ID игрока во избежание полного совпадения ФИО.

## Турниры

In [7]:
tournaments_df = pd.DataFrame().from_dict(pkl.load(open('tournaments.pkl', 'rb'))).T
tournaments_df.sample(4)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
2213,2213,Кубок ДоДона,2013-01-27T00:00:00+04:00,2013-01-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/44,[],None,"{'1': 15, '2': 15, '3': 15, '4': 15}"
654,654,Чемпионат Мордовии,2009-10-01T00:00:00+04:00,2010-05-30T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/9,[],None,None
6188,6188,Пятая актава: Тропік Казярога. Ліга нацый: Бел...,2020-01-09T18:00:00+03:00,2020-01-15T22:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 108971, 'name': 'Юлия', 'patronymic': ...",{'dateRequestsAllowedTo': '2020-01-15T20:00:00...,"{'1': 12, '2': 12, '3': 12}"
2890,2890,Чемпионат Финляндии,2014-05-10T00:00:00+04:00,2014-05-10T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/45,"[{'id': 5720, 'name': 'Иван', 'patronymic': 'В...",None,"{'1': 15, '2': 15, '3': 15}"


Информация о турнирах содержит данные об ID турнира и его названии, датах проведения. Есть какая-то информация о количестве вопросов (колонка `questionQty`); это можно будет провалидировать позднее.

## Результаты

Прочитать напрямую в `pandas.DataFrame` этот словарь не удалось, поскольку значения имеют разную длину. Для этого прочитаем данные по каждому ключу отдельно (не забыв сохранить этот ключ, так как по нему, судя по всему, необходимо будет производить слияние с таблицей турниров).

In [8]:
results = pkl.load(open('results.pkl', 'rb'))
df_list = []
for key, result in results.items():
    df_temp = pd.DataFrame().from_dict(result)
    df_temp['id'] = key
    df_list.append(df_temp)
results_df = pd.concat(df_list, ignore_index=True, sort=False)

In [9]:
results_df.sample(3)

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id
356341,"{'id': 49832, 'name': 'Судьба робота-собаки', ...",011111000111111100100111110101011001,"{'name': 'Калi капiтан - пацук', 'town': {'id'...",23.0,"{'id': 40466, 'venue': {'id': 3117, 'name': 'М...",43.5,[],[],"[{'flag': 'Л', 'usedRating': 13254, 'rating': ...",4663
152677,"{'id': 42178, 'name': 'Неопровержимые улитки',...",010100010101000110000011001100111000,"{'name': 'Неопровержимые улитки', 'town': {'id...",14.0,"{'id': 8260, 'venue': {'id': 3188, 'name': 'Ве...",202.5,[],[],"[{'flag': 'К', 'usedRating': 3996, 'rating': 4...",2783
323053,"{'id': 43516, 'name': 'Запасная хромосома', 't...",011000000000001000010000100100100000,"{'name': 'Запасная хромосома', 'town': {'id': ...",7.0,"{'id': 33397, 'venue': {'id': 3223, 'name': 'А...",98.0,"[{'id': 56139, 'questionNumber': 27, 'answer':...",[],"[{'flag': 'К', 'usedRating': 1585, 'rating': 1...",4354


Итак, в таблице результатов, помимо прочего, хранится информация о командах, маске (результаты ответа на вопросы), суммарное количество отвеченных вопросов (`questionsTotal`), позиция команды, предположительно данные об апелляциях (`controversials`) и ID турнира, по которому и будет производиться слияние.

В соответсвии с заданием, оставим только результаты, в которых есть информация об ответах команды и её составе.

In [10]:
(results_df['teamMembers'].apply(len) == 0).sum()

29589

In [11]:
results_df = results_df[(results_df['mask'].notna()) & (results_df['teamMembers'].apply(len) != 0)]

Перед разделением датасета на тренировочную и тестовую выборки сольём таблицы результатов и турниров.

In [12]:
final_df = pd.merge(
    results_df,
    tournaments_df,
    on='id'
)

In [13]:
final_df.sample(3)

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
303457,"{'id': 44786, 'name': '302-бис', 'town': {'id'...",101110011100101000110000000100111001,"{'name': '302-бис', 'town': {'id': 277, 'name'...",16.0,"{'id': 51262, 'venue': {'id': 3050, 'name': 'Р...",428.5,"[{'id': 83547, 'questionNumber': 14, 'answer':...",[],"[{'flag': None, 'usedRating': 6958, 'rating': ...",4981,ОВСЧ. 1 этап,2018-09-21T20:00:00+03:00,2018-09-25T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 505, 'name': 'Иделия', 'patronymic': '...",{'dateRequestsAllowedTo': '2018-09-25T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
120603,"{'id': 46573, 'name': 'Мятый элемент', 'town':...",000000001100010000000000000000000000,"{'name': 'Помятый бозон', 'town': {'id': 201, ...",3.0,"{'id': 22793, 'venue': {'id': 3117, 'name': 'М...",784.0,[],[],"[{'flag': 'Б', 'usedRating': 3631, 'rating': 3...",3329,Балтийский берег. 5 игра,2016-04-08T19:05:00+03:00,2016-04-12T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/49,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2016-04-10T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
302603,"{'id': 63442, 'name': 'Dream Team', 'town': {'...",010010001001111000100011101000000010,"{'name': 'Dream Team', 'town': {'id': 60, 'nam...",13.0,None,610.5,[],[],"[{'flag': 'Л', 'usedRating': 0, 'rating': 0, '...",4976,Школьный Синхрон-lite. Сезон 1,2017-11-17T18:00:00+03:00,2018-04-30T19:55:00+03:00,"{'id': 5, 'name': 'Общий зачёт'}",/seasons/51,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",None,"{'1': 36, '2': 36, '3': 36}"


Необходимые преобразования для колонок с датами:

In [14]:
for col in ['dateStart', 'dateEnd']:
    final_df[col] = pd.to_datetime(final_df[col], utc=True).dt.tz_localize(None)

In [15]:
final_df.sample(3)

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
344634,"{'id': 65626, 'name': 'Привкус победы', 'town'...",0110100100001011111101100010001100111111000011...,"{'name': 'Привкус победы', 'town': {'id': 197,...",30.0,None,17.0,[],[],"[{'flag': 'Б', 'usedRating': 3505, 'rating': 3...",5317,Щит и меч,2018-11-25 07:00:00,2018-11-25 12:00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 4715, 'name': 'Сергей', 'patronymic': ...",None,"{'1': 15, '2': 15, '3': 15, '4': 15}"
256440,"{'id': 41444, 'name': 'Полёт акваланга', 'town...",01101011010001X110111110111110110111,"{'name': 'Полёт акваланга', 'town': {'id': 201...",24.0,"{'id': 40727, 'venue': {'id': 3117, 'name': 'М...",4.0,"[{'id': 67909, 'questionNumber': 18, 'answer':...",[],"[{'flag': 'Л', 'usedRating': 10473, 'rating': ...",4494,Лига Сибири. 3 этап,2017-12-08 16:00:00,2017-12-12 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/51,"[{'id': 39218, 'name': 'Владислав', 'patronymi...",{'dateRequestsAllowedTo': '2017-12-12T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
386832,"{'id': 47284, 'name': 'Цианид калия', 'town': ...",001100000100110011001110000001001111,"{'name': 'Цианид калия', 'town': {'id': 21, 'n...",15.0,"{'id': 72975, 'venue': {'id': 3737, 'name': 'К...",640.5,[],"[{'id': 4, 'shortName': 'С', 'longName': 'Студ...","[{'flag': 'Б', 'usedRating': 4393, 'rating': 4...",5752,Балтийский Берег. 2 игра,2019-11-27 21:01:00,2019-12-04 20:55:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-12-04T19:00:00...,"{'1': 12, '2': 12, '3': 12}"


## Преобразования

Создадим колонку, в которой соберём ID игроков соответствующей команды.

In [16]:
final_df['playerIDs'] = final_df['teamMembers'].apply(lambda x: [player['player']['id'] for player in x])

In [17]:
final_df['team.id'] = final_df['team'].apply(lambda x: x.get('id', None))

In [18]:
final_df.sample(4)

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,playerIDs,team.id
164554,"{'id': 48807, 'name': 'Лаванда', 'town': {'id'...",000010000100100001100100000001110000000000000,"{'name': 'Лаванда', 'town': {'id': 161, 'name'...",9.0,None,25.0,[],"[{'id': 4, 'shortName': 'С', 'longName': 'Студ...","[{'flag': 'К', 'usedRating': 0, 'rating': 0, '...",3748,Этажи,2016-04-10 07:00:00,2016-04-10 15:00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/49,"[{'id': 40234, 'name': 'Александр', 'patronymi...",None,"{'1': 15, '2': 15, '3': 15}","[104088, 104089, 105575, 105576]",48807
179821,"{'id': 4228, 'name': 'Выпь', 'town': {'id': 27...",011100111101101111101011110111010100,"{'name': 'Выпь', 'town': {'id': 277, 'name': '...",24.0,"{'id': 26951, 'venue': {'id': 3050, 'name': 'Р...",101.5,[],[],"[{'flag': 'Б', 'usedRating': 7320, 'rating': 7...",3867,ОВСЧ. 2 этап,2016-10-21 16:00:00,2016-10-25 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/50,"[{'id': 505, 'name': 'Иделия', 'patronymic': '...",{'dateRequestsAllowedTo': '2016-10-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[7231, 133, 40931, 17343, 55585, 128161]",4228
434664,"{'id': 74645, 'name': 'Молито', 'town': {'id':...",000000001001000000001000000000000000,"{'name': 'Молито', 'town': {'id': 2163, 'name'...",3.0,None,765.0,[],"[{'id': 2, 'shortName': 'МШ', 'longName': 'Мла...","[{'flag': None, 'usedRating': 0, 'rating': 0, ...",6249,Школьный синхрон-lite. Сезон 3,2019-08-31 21:05:00,2020-04-30 20:55:00,"{'id': 5, 'name': 'Общий зачёт'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",None,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ...","[210348, 210349, 210350, 210351, 210352, 210353]",74645
225011,"{'id': 58315, 'name': 'Оливковая эйфория', 'to...",001001000000001000000000000000001000,"{'name': 'Оливковая эйфория', 'town': {'id': 2...",4.0,None,27.5,[],[],"[{'flag': 'К', 'usedRating': 0, 'rating': 0, '...",4326,Межфакультетский кубок МГУ. Отбор № 6,2017-04-07 11:00:00,2017-04-07 11:00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/50,"[{'id': 5990, 'name': 'Андрей', 'patronymic': ...",None,"{'1': 12, '2': 12, '3': 12}","[146976, 146977, 146978, 146979, 146980]",58315


Для ускорения работы и экономии памяти оставим только турниры из 2019 и 2020 годов. Построим таблицу со следующими строками: (ID игрока, ID вопроса, результат)

In [19]:
short_df = final_df.query('dateStart < "2021" and dateStart >= "2019"')[
    ['mask', 'dateStart', 'id', 'playerIDs', 'team.id']
]

In [20]:
short_df.sample(3)

,mask,dateStart,id,playerIDs,team.id
361846,001011010001000100001011011010100010001,2019-04-01 11:00:00,5545,"[31070, 27441, 8896, 71506, 41846, 71507]",41420
316650,111110101011111110101101100100000000000001000010,2019-03-29 16:00:00,5025,"[37572, 33096, 29805, 36274, 36920, 45344]",3883
397327,1000101000110011100100010000001000001110100000...,2019-10-20 09:00:00,5795,"[24365, 74181, 69503, 8103, 39036, 10774]",575


In [21]:
short_df['questionIDs'] = None

In [22]:
short_df['id'] = short_df['id'].astype(str)

In [23]:
for idx, row in short_df.iterrows():
    short_df.loc[idx, 'questionIDs'] = str(list((row['id']+'_'+str(i), row['mask'][i]) for i in range(len(row['mask']))))

In [24]:
short_df['questionIDs'] = short_df['questionIDs'].apply(eval)

In [25]:
short_df.sample(3)

,mask,dateStart,id,playerIDs,team.id,questionIDs
432972,111100001010011100100111110100111001,2020-03-05 16:00:00,6218,"[55531, 96325, 5130, 18070, 219893]",50869,"[(6218_0, 1), (6218_1, 1), (6218_2, 1), (6218_..."
301155,110001000101111011100111111110111110,2019-04-05 16:05:00,4975,"[92909, 126413, 94793, 61311, 12456, 15501]",51461,"[(4975_0, 1), (4975_1, 1), (4975_2, 0), (4975_..."
405523,010001001011000010111001101001001100,2020-02-12 21:00:00,5823,"[32749, 53524, 93874, 62383]",27477,"[(5823_0, 0), (5823_1, 1), (5823_2, 0), (5823_..."


In [26]:
short_df = short_df.explode(column='playerIDs', ignore_index=True)

In [27]:
short_df.sample(4)

,mask,dateStart,id,playerIDs,team.id,questionIDs
345452,111110101000011011101111011100110010,2020-03-07 13:00:00,5799,25177,4109,"[(5799_0, 1), (5799_1, 1), (5799_2, 1), (5799_..."
491250,110100100001100110101001001000010010,2019-11-16 09:00:00,6071,160662,27656,"[(6071_0, 1), (6071_1, 1), (6071_2, 0), (6071_..."
93848,1111111101101111111001001110111011100000111101...,2019-02-16 09:30:00,5325,2857,4989,"[(5325_0, 1), (5325_1, 1), (5325_2, 1), (5325_..."
106430,010010100001101010100010010100011100,2019-02-14 21:00:00,5390,165619,8015,"[(5390_0, 0), (5390_1, 1), (5390_2, 0), (5390_..."


In [28]:
short_df = short_df.explode(column='questionIDs', ignore_index=True)

In [29]:
short_df['question_result'] = short_df['questionIDs'].str[-1]
short_df['questionID'] = short_df['questionIDs'].str[0]

In [30]:
short_df.sample(3)

,mask,dateStart,id,playerIDs,team.id,questionIDs,question_result,questionID
5596337,110000110100100000000111010010000000,2020-04-18 16:00:00,5477,171960,59920,"(5477_14, 0)",0,5477_14
22022146,111101101111111111111110011111011111110011111111,2020-02-29 08:00:00,6233,69394,4109,"(6233_33, 1)",1,6233_33
5548044,1110100110010010100100011000001101110000110011...,2019-04-13 00:00:00,5475,126814,70713,"(5475_87, 0)",0,5475_87


## Разделение данных

In [31]:
cols_to_leave = ['team.id', 'playerIDs', 'questionID', 'question_result']
train_df = short_df.loc[short_df['dateStart'].dt.year == 2019, cols_to_leave]
test_df = short_df.loc[short_df['dateStart'].dt.year == 2020, cols_to_leave]

In [32]:
train_df.query('question_result == "X"').head()

,team.id,playerIDs,questionID,question_result
619526,69309,27822,4986_23,X
619562,69309,28751,4986_23,X
619598,69309,30270,4986_23,X
619634,69309,33620,4986_23,X
619670,69309,2421,4986_23,X


In [33]:
train_df.sample(3)

,team.id,playerIDs,questionID,question_result
17669591,67543,125697,5890_15,0
22252574,56369,183736,6249_163,0
693329,56373,138370,4986_26,1


В тренировочной выборке имеются вопросы, отмеченные как `X`. Предположительно, данные вопросы были сняты с турнира. Данное предположение подтверждается, что с помощью `X` отмечены одинаковые вопросы для всех команд. Пример:

In [34]:
final_df.query("id == 4986")

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,playerIDs,team.id
307844,"{'id': 69309, 'name': 'Брют', 'town': {'id': 2...",11111111111111110111101X111111111111,"{'name': 'Брют', 'town': {'id': 201, 'name': '...",33.0,"{'id': 58884, 'venue': {'id': 3117, 'name': 'М...",1.0,[],[],"[{'flag': None, 'usedRating': 14484, 'rating':...",4986,ОВСЧ. 6 этап,2019-02-15 17:00:00,2019-02-19 17:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[27822, 28751, 30270, 33620, 2421]",69309
307845,"{'id': 49758, 'name': 'Мимино', 'town': {'id':...",11111111011111110111111X110111111111,"{'name': 'Мимино', 'town': {'id': 449, 'name':...",32.0,"{'id': 58921, 'venue': {'id': 3117, 'name': 'М...",2.0,[],[],"[{'flag': 'Л', 'usedRating': 12749, 'rating': ...",4986,ОВСЧ. 6 этап,2019-02-15 17:00:00,2019-02-19 17:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[6482, 13782, 33509, 34936, 27240, 36754]",49758
307846,"{'id': 312, 'name': 'Социал-демократы', 'town'...",11111110011111011111011X111111111111,"{'name': 'Социал-демократы', 'town': {'id': 20...",31.0,"{'id': 58884, 'venue': {'id': 3117, 'name': 'М...",3.5,"[{'id': 94154, 'questionNumber': 18, 'answer':...",[],"[{'flag': 'Л', 'usedRating': 11577, 'rating': ...",4986,ОВСЧ. 6 этап,2019-02-15 17:00:00,2019-02-19 17:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[19411, 19599, 25724, 9678, 5894, 32266]",312
307847,"{'id': 27177, 'name': 'Призраки Коши', 'town':...",11111110111111011111011X111111101111,"{'name': 'Призраки Коши', 'town': {'id': 285, ...",31.0,"{'id': 56414, 'venue': {'id': 3030, 'name': 'С...",3.5,[],[],"[{'flag': 'Л', 'usedRating': 13667, 'rating': ...",4986,ОВСЧ. 6 этап,2019-02-15 17:00:00,2019-02-19 17:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[6212, 37718, 16837, 33032, 31493, 30371]",27177
307848,"{'id': 928, 'name': 'ОМ', 'town': {'id': 67, '...",11111111111111110101011X110101111111,"{'name': 'ОМ', 'town': {'id': 67, 'name': 'Вор...",30.0,"{'id': 58770, 'venue': {'id': 3065, 'name': 'В...",6.5,"[{'id': 94319, 'questionNumber': 23, 'answer':...",[],"[{'flag': 'К', 'usedRating': 10074, 'rating': ...",4986,ОВСЧ. 6 этап,2019-02-15 17:00:00,2019-02-19 17:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[5248, 48319, 26101, 82868, 37746, 36910]",928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308746,"{'id': 69290, 'name': 'Исполкони', 'town': {'i...",00000000000000000000000X000000000000,"{'name': 'Исполкони', 'town': {'id': 22, 'name...",0.0,"{'id': 57006, 'venue': {'id': 3237, 'name': 'Б...",905.5,[],[],"[{'flag': 'Б', 'usedRating': 0, 'rating': 0, '...",4986,ОВСЧ. 6 этап,2019-02-15 17:00:00,2019-02-19 17:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}","[190528, 190529, 191319, 191320]",69290
308747,"{'id': 69437, 'name': 'АКМ', 'town': {'id': 16...",00000000000000000000000X000000000000,"{'name': 'АКМ', 'town': {'id': 1693, 'name': '...",0.0,"{'id': 58942, 'venue': {'id': 3303, 'name': 'А...",905.5,[],[],"[{'flag': 'Л', 'usedRating': 3653, 'rating': 3...",4986,ОВСЧ. 6 этап,2019-02-15 17:00:00,2019-02-19 17:00:00,"{'id': 3, 'name': 'Синхрон'}",/

Таким образом, данные вопросы следует удалить и из тренировочной, и из тестовой выборки.

In [35]:
final_df.query("id == 5141").sample(10)

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,playerIDs,team.id
332947,"{'id': 48997, 'name': 'Уэллс Фарго', 'town': {...",1100000010011011010100011001010101X0110010100011,"{'name': 'Уэллс Фарго', 'town': {'id': 161, 'n...",21.0,"{'id': 57408, 'venue': {'id': 3077, 'name': 'К...",447.0,"[{'id': 92021, 'questionNumber': 7, 'answer': ...",[],"[{'flag': 'К', 'usedRating': 8339, 'rating': 8...",5141,Азовский бриз,2019-01-18 16:00:00,2019-01-22 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 15755, 'name': 'Александр', 'patronymi...",{'dateRequestsAllowedTo': '2019-01-22T23:59:59...,"{'1': 12, '2': 12, '3': 12, '4': 12}","[118683, 133861, 119947, 110409, 118681, 189514]",48997
332650,"{'id': 1025, 'name': 'Двин', 'town': {'id': 10...",1110110110111011100101110100000101X1111011101111,"{'name': 'Двин', 'town': {'id': 100, 'name': '...",31.0,"{'id': 57308, 'venue': {'id': 3033, 'name': 'Е...",164.0,[],[],"[{'flag': 'Б', 'usedRating': 8701, 'rating': 8...",5141,Азовский бриз,2019-01-18 16:00:00,2019-01-22 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 15755, 'name': 'Александр', 'patronymi...",{'dateRequestsAllowedTo': '2019-01-22T23:59:59...,"{'1': 12, '2': 12, '3': 12, '4': 12}","[12915, 658, 8118, 19143, 42837]",1025
332681,"{'id': 55501, 'name': 'Семь сорок', 'town': {'...",1111011110111001110110011111001110X1110010010110,"{'name': 'Семь сорок', 'town': {'id': 197, 'na...",31.0,"{'id': 57551, 'venue': {'id': 3112, 'name': 'М...",164.0,[],[],"[{'flag': 'Б', 'usedRating': 8456, 'rating': 8...",5141,Азовский бриз,2019-01-18 16:00:00,2019-01-22 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 15755, 'name': 'Александр', 'patronymi...",{'dateRequestsAllowedTo': '2019-01-22T23:59:59...,"{'1': 12, '2': 12, '3': 12, '4': 12}","[6284, 29049, 10128, 25875, 10989]",55501
332671,"{'id': 46953, 'name': 'Molinos', 'town': {'id'...",1111111110010011100100011101100110X0110111111110,"{'name': 'Molinos', 'town': {'id': 285, 'name'...",31.0,"{'id': 56942, 'venue': {'id': 3030, 'name': 'С...",164.0,[],[],"[{'flag': 'Б', 'usedRating': 6709, 'rating': 6...",5141,Азовский бриз,2019-01-18 16:00:00,2019-01-22 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 15755, 'name': 'Александр', 'patronymi...",{'dateRequestsAllowedTo': '2019-01-22T23:59:59...,"{'1': 12, '2': 12, '3': 12, '4': 12}","[98598, 115148, 98599, 162691, 125018, 134358]",46953
332611,"{'id': 26656, 'name': 'Ни стыда, ни совести', ...",1111111000111001110110011111100100X1111111100111,"{'name': 'Ни стыда, ни совести', 'town': {'id'...",33.0,"{'id': 57408, 'venue': {'id': 3077, 'name': 'К...",109.5,"[{'id': 92062, 'questionNumber': 32, 'answer':...",[],"[{'flag': 'Б', 'usedRating': 10102, 'rating': ...",5141,Азовский бриз,2019-01-18 16:00:00,2019-01-22 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 15755, 'name': 'Александр', 'patronymi...",{'dateRequestsAllowedTo': '2019-01-22T23:59:59...,"{'1': 12, '2': 12, '3': 12, '4': 12}","[17146, 38047, 88067, 31231, 73295, 101697]",26656
332568,"{'id': 59580, 'name': 'И остались голодными', ...",1111110111111011110110011111110010X1100010111111,"{'name': 'И остались голодными', 'town': {'id'...",35.0,"{'id': 57468, 'venue': {'id': 3133, 'name': 'С...",58.5,[],[],"[{'flag': 'Л', 'usedRating': 11455, 'rating': ...",5141,Азовский бриз,2019-01-18 16:00:00,2019-01-22 16:00:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 15755, 'name': 'Александр', 'patronymi...",{'dateRequestsAllowedTo': '2019-01-22T23:59:59...,"{'1': 12, '2': 12, '3': 12, '4': 12}","[27009, 116628, 105369, 10236, 88505, 54574]",59580
332869,"{'id': 55924, 'name': 'Почему лев царь зверей?...",1110110100110101100100011111000100X1110010101100,"{'name': '6 персонажей в поисках Сартра', 'tow...",25.0,"{'id': 57372, 'venue': {'id': 3103, 'name': 'И...",356.5,[],[],"[{'flag': 'Л', 'usedRating': 9965, 'rating': 9...

Также стоит удалить записи с `?` в качестве результата. Предполагаю, что это нерешенная апелляция или её незарегистрированный результат.

In [36]:
train_df['question_result'].unique()

array(['1', '0', 'X', '?'], dtype=object)

In [37]:
train_df = train_df.query('question_result != "X" and question_result != "?"').astype({'question_result': int})
test_df = test_df.query('question_result != "X" and question_result != "?"').astype({'question_result': int})

In [38]:
train_df.sample(3)

,team.id,playerIDs,questionID,question_result
6532704,56625,142677,5534_9,0
18059111,7567,15466,5920_30,0
9288645,67142,118683,5698_3,0


## OneHot-кодирование и тренировка baseline-модели

In [39]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [40]:
train_ohe = ohe.fit_transform(train_df.drop(columns=['team.id', 'question_result']))

In [41]:
logreg = LogisticRegression(solver='saga', fit_intercept=True)

In [42]:
logreg.fit(train_ohe, train_df['question_result'])

LogisticRegression(solver='saga')

In [43]:
pkl.dump(logreg, open('logreg-fit-intercept.pkl', 'wb'))

In [44]:
logreg.intercept_

array([-1.63236333])

В качестве результата возьмем веса натренированной модели.

In [45]:
weight_df = pd.DataFrame({'id': ohe.categories_[0], 'weight': logreg.coef_[0][:len(ohe.categories_[0])]}).sort_values(by='weight', ascending=False)

In [46]:
weight_df = pd.merge(weight_df, players_df, how='left')

In [47]:
weight_df.head(10)

,id,weight,name,patronymic,surname
0,27403,4.105632,Максим,Михайлович,Руссо
1,4270,3.978506,Александра,Владимировна,Брутер
2,28751,3.952230,Иван,Николаевич,Семушин
3,30152,3.775838,Артём,Сергеевич,Сорожкин
4,27822,3.770671,Михаил,Владимирович,Савченков
5,30270,3.767216,Сергей,Леонидович,Спешков
6,20691,3.624435,Станислав,Григорьевич,Мереминский
7,18036,3.622061,Михаил,Ильич,Левандовский
8,26089,3.580913,Ирина,Сергеевна,Прокофьева
9,22799,3.572561,Сергей,Игоревич,Николенко


## Оценка качества

In [48]:
test_df_ = final_df[final_df['dateStart'].dt.year==2020]

Создаем отображение из ID игрока на его рейтинг.

In [49]:
mapping = dict()
for idx, row in weight_df.iterrows():
    mapping[row['id']] = row['weight']

В качестве самого простой статистики для оценки рейтинга можно взять сумму всех рейтингов игроков, умноженную на -1 (чем выше рейтинг, тем ниже номер в рейтинговой таблице).

In [50]:
test_df_['team_rating'] = test_df_['playerIDs'].apply(lambda row: -sum(mapping.get(id_, logreg.intercept_[0]) for id_ in row))

C:\Users\nickp\AppData\Local\Temp\ipykernel_8484\4025077547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_['team_rating'] = test_df_['playerIDs'].apply(lambda row: -sum(mapping.get(id_, logreg.intercept_[0]) for id_ in row))


Отсортируем значения

In [51]:
test_df_.sort_values(by=['id', 'position'], inplace=True)

C:\Users\nickp\AppData\Local\Temp\ipykernel_8484\3152601698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_.sort_values(by=['id', 'position'], inplace=True)


In [52]:
test_df_.groupby('id').apply(lambda group: spearmanr(group['position'], group['team_rating'])[0]).mean()

0.7907635288779833

In [53]:
test_df_.groupby('id').apply(lambda group: kendalltau(group['position'], group['team_rating'])[0]).mean()

0.6306975038260684

Таким образом, достаточно простая модель, выдает 0,79 по корреляции Спирмана и 0,63 по корреляции Кендалла, что вполне себе неплохой результат.

## Рейтинг-лист турниров

In [54]:
question_df = pd.DataFrame({'question_id': ohe.categories_[1], 'weight': logreg.coef_[0][len(ohe.categories_[0]):]}).sort_values(by='weight', ascending=False)

Рейтинг у вопросов обратный, так как больший вес приводит к большей вероятности на него ответить, верно также обратное.

In [55]:
question_df.tail(10)

,question_id,weight
21746,5795_8,-5.340594
21734,5795_50,-5.340594
21726,5795_43,-5.340595
20827,5774_6,-5.360813
21085,5780_11,-5.506795
1763,5159_34,-5.540932
1735,5159_0,-5.540934
8391,5465_67,-5.579494
8360,5465_39,-5.579495
2023,5186_17,-5.711728


Из рейтинга топ-10 вопросов 2019-го года удалось найти следующие:
* [Первенство правого полушария, вопрос 35](https://db.chgk.info/node/8307)
* [Первенство правого полушария, вопрос 1](https://db.chgk.info/node/8307)
* [Кубок ярмарок, вопрос 12](https://db.chgk.info/node/8070)
* [Кубок Москвы, вопрос 8](https://db.chgk.info/node/8376)
* [Кубок Москвы, вопрос 9](https://db.chgk.info/node/8376)
* [Кубок Москвы, вопрос 44](https://db.chgk.info/node/8376)
* [Кубок Москвы, вопрос 51](https://db.chgk.info/node/8376)

In [56]:
question_df['id'] = question_df['question_id'].str.split('_').str[0]

In [57]:
tour_rating_df = question_df.groupby('id', as_index=False)['weight'].sum()

In [58]:
tour_rating_df['id'] = tour_rating_df['id'].astype(int)

In [59]:
tournaments_df['id'] = tournaments_df['id'].astype(int)

In [60]:
tournaments_df.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [61]:
tour_rating_df.merge(tournaments_df[['id', 'name']], on='id').sort_values(by='weight')

,id,weight,name
665,6149,-800.431321,Чемпионат Санкт-Петербурга. Первая лига
666,6150,-202.673013,Чемпионат Санкт-Петербурга. Высшая лига
179,5465,-139.270284,Чемпионат России
632,6085,-133.496634,Серия Гран-при. Общий зачёт
543,5928,-103.008129,Угрюмый Ёрш
...,...,...,...
637,6090,133.668571,Дзержинский марафон
289,5592,144.105423,Студенческая лига ЧТ
469,5827,323.016336,Шестой киевский марафон. Асинхрон
674,6249,410.151888,Школьный синхрон-lite. Сезон 3


Удивительно, что Чемпионаты Санкт-Петербурга так далеко отстоят от остальных турниров. Сложно пока сказать, есть ли этому численное объяснение. Возможно, имела бы смысл нормировка рейтинга, когда вместо суммы рейтингов стоило бы взять их среднее.

## EM-алгоритм

В качестве скрытой переменной можно попробовать использовать вероятность того, что игрок ответил на вопрос при условии, что команда ответила на вопрос. Эту вероятность мы и будем пересчитывать на Е-шаге и использовать как целевую переменную при моделировании на M-шаге, например, с помощью логистической регрессии. 

На Е-шаге будем нормировать переменные: если команда не ответила на вопрос, то соответствующие скрытые переменные зануляем; если ответила и значения скрытых переменных для команды равны $z_1, z_2, ..., z_k$, то новое значение будет равно: $$\tilde{z}_i=\frac{z_i}{1 - \prod_{j=1}^k(1-z_j)}$$

Так как надо будет учиться не на бинарной целевой переменной, нужно немного преобразовать логистическую регрессию.

In [77]:
class FractionalLogisticRegression(LogisticRegression):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fitted = False
        
    def fit_(self, X, y, lr=1):
        if not self.fitted:
            super().fit(X, y)
            self.fitted = True
        X_transpose = X.T
        
        for i in range(self.max_iter):
            preds = self.predict_proba(X)[:, 1]
            grads = X_transpose @ (preds - y) / X.shape[0]
            
            self.coef_[0] -= lr * grads
            
            if i > 0 and i % 10 == 0:
                lr /= 2
            
        

In [78]:
flogreg = FractionalLogisticRegression(solver='saga', tol=0.05)

In [65]:
del test_df, final_df, players_df, tour_rating_df, tournaments_df, question_df, results_df, results, result, short_df, logreg

In [79]:
z = train_df['question_result']  # logreg.predict_proba(train_ohe)[:, 1]
n_iter = 10
for i in range(n_iter):
    print(f"Iter #{i}")
    # E-шаг. Обновляем значения скрытых переменных
    print("\tE-step")
    train_df['z'] = z
    train_df['comp_z'] = 1 - z
    train_df['team_score'] = 1 - train_df.groupby(['team.id', 'questionID'])['comp_z'].transform(np.prod)
    z = np.where(train_df['question_result'] == 0, 1e-5, train_df['z'] / train_df['team_score'])
    
    # M-шаг
    print("\tM-step")
    flogreg.fit_(train_ohe, z)
    z = flogreg.predict_proba(train_ohe)[:, 1]
    
    # Оценка модели
    weight_df = pd.DataFrame(
        {
            'id': ohe.categories_[0], 
            'weight': flogreg.coef_[0][:len(ohe.categories_[0])]
        }
    ).sort_values(by='weight', ascending=False)
    mapping = dict()
    for idx, row in weight_df.iterrows():
        mapping[row['id']] = row['weight']
    test_df_['team_rating'] = test_df_['playerIDs'].apply(lambda row: -sum(mapping.get(id_, flogreg.intercept_[0]) for id_ in row))
    test_df_.sort_values(by=['id', 'position'], inplace=True)
    print(f"\tSpearman corr: {test_df_.groupby('id').apply(lambda group: spearmanr(group['position'], group['team_rating'])[0]).mean():.4f}")
    print(f"\tKendall corr: {test_df_.groupby('id').apply(lambda group: kendalltau(group['position'], group['team_rating'])[0]).mean():.4f}")

Iter #0
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6306
Iter #1
	E-step
	M-step
	Spearman corr: 0.7907
	Kendall corr: 0.6306
Iter #2
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6306
Iter #3
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6306
Iter #4
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6306
Iter #5
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6306
Iter #6
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6306
Iter #7
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6305
Iter #8
	E-step
	M-step
	Spearman corr: 0.7906
	Kendall corr: 0.6305
Iter #9
	E-step
	M-step
	Spearman corr: 0.7896
	Kendall corr: 0.6298
